In [126]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import pandas as pd
import numpy as np
import json
from ssl import create_default_context
from datetime import datetime
from datetime import timedelta
import requests
#from getpass import getpass
import parametros
import time

## Conectando a ElasticSearch

In [127]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
)

### tomando fecha mas reciente del indice

In [128]:
'''total_docs = 0
try:
    response = es.search(
        index= parametros.cambium_d_d_index,
        body={"aggs" : {
                   "max_date": {"max": {"field": "registration_date", "format": "yyyy-MM-dd HH:mm:ss"}}
                }
             },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["aggregations"]
    fecha_max=response["aggregations"]["max_date"]['value_as_string']
except:
    fecha_max = '2010-01-01 00:00:01'
print("ultima fecha en indice: ",fecha_max)'''

'total_docs = 0\ntry:\n    response = es.search(\n        index= parametros.cambium_d_d_index,\n        body={"aggs" : {\n                   "max_date": {"max": {"field": "registration_date", "format": "yyyy-MM-dd HH:mm:ss"}}\n                }\n             },\n        size=total_docs\n    )\n    #print(es.info())\n    elastic_docs = response["aggregations"]\n    fecha_max=response["aggregations"]["max_date"][\'value_as_string\']\nexcept:\n    fecha_max = \'2010-01-01 00:00:01\'\nprint("ultima fecha en indice: ",fecha_max)'

In [129]:
now = datetime.now()
new_date = now - timedelta(days=6)
format = "%Y%m%d"
format_ES = "%Y.%m.%d"
fecha_url = str(new_date.strftime(format))
fecha_hoy = str(now.strftime(format_ES))

## Leyendo la APi cambium-devices

In [130]:
# Definimos la cabecera y el diccionario con los datos
datos_api = pd.DataFrame(columns=["country","description","name","ip","network","product","msn"
                                  #,"software_version"
                                  #,"inactive_software_version","last_reboot_reason","hardware_version"
                                  ,"registration_date","status","type","tower","site","site_id"
                                          ,"location.type","location.coordinates","mac","managed_account","status_time"
                                          ,"config.version","config.variables.DEFAULT_GW","config.variables.DEFAULT_GW6"
                                          ,"config.variables.DNS_SERVER6_1","config.variables.DNS_SERVER6_2"
                                          ,"config.variables.DNS_SERVER_1","config.variables.DNS_SERVER_2"
                                          ,"config.variables.VLAN_1_IP","config.variables.VLAN_1_IP6"
                                          ,"config.variables.VLAN_1_MASK","config.variables.VLAN_1_MODE"
                                          ,"config.variables.VLAN_1_MODE6","config.variables.VLAN_1_PREFIX6"
                                          ,"config.variables.LOCATION","config.variables.SHUTDOWN_24"
                                          ,"config.variables.CHANNEL_24","config.variables.SHUTDOWN_5"
                                          ,"config.variables.CHANNEL_5","config.variables.POWER_24"
                                          ,"config.variables.POWER_5","config.variables.DISPLAY_NAME","ap_group"]
                                 )
j=0
estados = ['online','offline','onboarding']
for k in range(0,len(parametros.url_cambium)):
    print("servidor:",k+1)
    token_aux = 'Bearer ' + parametros.cambium_token_aux[k]
    cabecera1 = { 'Content-Type': 'application/json', 'Authorization' : token_aux, 'accept' : '*/*' } 
    for m in range(0,len(estados)):
        url = parametros.url_cambium[k] + "devices" + "?status=" + estados[m] #'?status=online'
        print(url)
        r = requests.get(url, headers = cabecera1, verify=False)
        if r.status_code == 200:
            res = json.loads(r.text)
            time.sleep(4)
            dato_param = res['paging']
            #se calcula los ciclos de la consulta paginada
            ciclos = int(round(dato_param['total']/100,0))
            #ciclos = 0 #int(round(dato_param['total']/100,0))
            #print(dato_param['total'])
            i = 0
            while i <= ciclos:
                offset = str(i*100)
                #url2 = parametros.cambium_devices_url + '?offset=' + offset
                url2 = url + '&offset=' + offset 
                i+=1
                #print(url2)
                r = requests.get(url2, headers = cabecera1, verify=False)
                if r.status_code == 200:
                    #print("respuesta 200: ",r.text)
                    res = json.loads(r.text)
                    for x in range(0,len(res['data'])):
                        #print(res['data'][x]['names'])
                        try:
                            country = res['data'][x]['country']
                        except:
                            country = ''
                        try:
                            description = res['data'][x]['description']
                        except:
                            description = ''
                        try:
                            tower = res['data'][x]['tower']
                        except:
                            tower = ''
                        try:
                            site = res['data'][x]['site']
                        except:
                            site = ''
                        try:
                            site_id = res['data'][x]['site_id']
                        except:
                            site_id = ''
                        try:
                            status_time = res['data'][x]['status_time']
                        except:
                            status_time = ''
                        try:
                            ap_group = res['data'][x]['ap_group']
                        except:
                            ap_group = ''
                        try:
                            config = res['data'][x]['config']
                        #try:
                        #    software_version = res['data'][x]['software_version']
                        #except:
                             #software_version = ''
                            if config['version']== '1':
                                version = '1'
                                DEFAULT_GW=''
                                DEFAULT_GW6=''
                                DNS_SERVER6_1=''
                                DNS_SERVER6_2=''
                                DNS_SERVER_1=''
                                DNS_SERVER_2=''
                                VLAN_1_IP=''
                                VLAN_1_IP6=''
                                VLAN_1_MASK=''
                                VLAN_1_MODE=''
                                VLAN_1_MODE6=''
                                VLAN_1_PREFIX6=''
                                LOCATION=''
                                SHUTDOWN_24=''
                                CHANNEL_24=''
                                SHUTDOWN_5=''
                                CHANNEL_5=''
                                POWER_24=''
                                POWER_5=''
                                DISPLAY_NAME=''
                            else:
                                version = config['variables']
                                DEFAULT_GW= config['variables']['DEFAULT_GW']
                                DEFAULT_GW6= config['variables']['DEFAULT_GW6']
                                DNS_SERVER6_1= config['variables']['DNS_SERVER6_1']
                                DNS_SERVER6_2= config['variables']['DNS_SERVER6_2']
                                DNS_SERVER_1= config['variables']['DNS_SERVER_1']
                                DNS_SERVER_2= config['variables']['DNS_SERVER_2']
                                VLAN_1_IP= config['variables']['VLAN_1_IP']
                                VLAN_1_IP6= config['variables']['VLAN_1_IP6']
                                VLAN_1_MASK= config['variables']['VLAN_1_MASK']
                                VLAN_1_MODE= config['variables']['VLAN_1_MODE']
                                VLAN_1_MODE6= config['variables']['VLAN_1_MODE6']
                                VLAN_1_PREFIX6= config['variables']['VLAN_1_PREFIX6']
                                LOCATION= config['variables']['LOCATION']
                                SHUTDOWN_24= config['variables']['SHUTDOWN_24']
                                CHANNEL_24= config['variables']['CHANNEL_24']
                                SHUTDOWN_5= config['variables']['SHUTDOWN_5']
                                CHANNEL_5= config['variables']['CHANNEL_5']
                                POWER_24= config['variables']['POWER_24']
                                POWER_5= config['variables']['POWER_5']
                                DISPLAY_NAME= config['variables']['DISPLAY_NAME']
                        except:
                            version = ''
                            DEFAULT_GW=''
                            DEFAULT_GW6=''
                            DNS_SERVER6_1=''
                            DNS_SERVER6_2=''
                            DNS_SERVER_1=''
                            DNS_SERVER_2=''
                            VLAN_1_IP=''
                            VLAN_1_IP6=''
                            VLAN_1_MASK=''
                            VLAN_1_MODE=''
                            VLAN_1_MODE6=''
                            VLAN_1_PREFIX6=''
                            LOCATION=''
                            SHUTDOWN_24=''
                            CHANNEL_24=''
                            SHUTDOWN_5=''
                            CHANNEL_5=''
                            POWER_24=''
                            POWER_5=''
                            DISPLAY_NAME=''
                        datos_api.loc[j] = [country
                                            ,description
                                            ,res['data'][x]['name']
                                            ,res['data'][x]['ip']
                                            ,res['data'][x]['network']
                                            ,res['data'][x]['product']
                                            ,res['data'][x]['msn']
                                            #, software_version #,res['data'][x]['software_version']
                                            #,res['data'][x]['inactive_software_version']
                                            #,res['data'][x]['last_reboot_reason']
                                            #,res['data'][x]['hardware_version']
                                            ,res['data'][x]['registration_date']
                                            ,res['data'][x]['status']
                                            ,res['data'][x]['type']
                                            ,tower
                                            ,site
                                            ,site_id
                                            ,res['data'][x]['location']['type']
                                            ,res['data'][x]['location']['coordinates']
                                            ,res['data'][x]['mac']
                                            ,res['data'][x]['managed_account']
                                            ,status_time
                                            ,version
                                            ,DEFAULT_GW
                                            ,DEFAULT_GW6
                                            ,DNS_SERVER6_1
                                            ,DNS_SERVER6_2
                                            ,DNS_SERVER_1
                                            ,DNS_SERVER_2
                                            ,VLAN_1_IP
                                            ,VLAN_1_IP6
                                            ,VLAN_1_MASK
                                            ,VLAN_1_MODE
                                            ,VLAN_1_MODE6
                                            ,VLAN_1_PREFIX6
                                            ,LOCATION
                                            ,SHUTDOWN_24
                                            ,CHANNEL_24
                                            ,SHUTDOWN_5
                                            ,CHANNEL_5
                                            ,POWER_24
                                            ,POWER_5
                                            ,DISPLAY_NAME
                                            ,ap_group]
                        j+=1 # es el indice  
                        
                else:
                    print("error bucle interno:",r.status_code)
                    #break
                print (url2)    
                print("bucle:",i)    
                time.sleep(5)    
        else:
            print("error bucle externo:",r.status_code)
            #break

servidor: 1
https://100.123.26.252/api/v2/devices?status=online


/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.252'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.252'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://100.123.26.252/api/v2/devices?status=online&offset=0
bucle: 1


/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.252'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://100.123.26.252/api/v2/devices?status=online&offset=100
bucle: 2


...

https://100.123.26.224/api/v2/devices?status=onboarding&offset=100
bucle: 2


/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.224'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://100.123.26.224/api/v2/devices?status=onboarding&offset=200
bucle: 3


/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.224'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://100.123.26.224/api/v2/devices?status=onboarding&offset=300
bucle: 4


### Borrado de filas con todos los valorar NaN

In [131]:
datos_api

,country,description,name,ip,network,product,msn,registration_date,status,type,...,config.variables.VLAN_1_PREFIX6,config.variables.LOCATION,config.variables.SHUTDOWN_24,config.variables.CHANNEL_24,config.variables.SHUTDOWN_5,config.variables.CHANNEL_5,config.variables.POWER_24,config.variables.POWER_5,config.variables.DISPLAY_NAME,ap_group
0,CO,None,44745-ZGYO227-AP-INT,172.28.118.179,SANTANDER,cnPilot e410,W8WJ037WHH74,2021-04-07T19:13:54-05:00,online,wifi-enterprise,...,,,,,,,,,,INDOOR-44745-ZGYO227
1,CO,None,44745-ZGYO227-AP-EXT2,172.28.118.180,SANTANDER,cnPilot e510,W8VJ067FJH0D,2021-04-07T19:38:50-05:00,online,wifi-enterprise,...,,,,,,,,,,OUTDOOR
2,CO,None,44745-ZGYO227-AP-EXT1,172.28.118.181,SANTANDER,cnPilot e510,W8VJ05X88C78,2021-04-08T10:50:15-05:00,online,wifi-enterprise,...,,,,,,,,,,OUTDOOR
3,CO,44747-ZGYO413-AP-INT,44747-ZGYO413-AP-INT,172.28.118.171,SANTANDER,cnPilot e410,W8WK06VL2ZGJ,2021-04-19T10:46:35-05:00,online,wifi-enterprise,...,,,,,,,,,,INDOOR_44747-ZGYO413
4,CO,44747-ZGYO413-AP-EXT-1,44747-ZGYO413-AP-EXT-1,172.28.118.172,SANTANDER,cnPilot e510,W8WF00HRR3VK,2021-04-19T10:46:34-05:00,online,wifi-enterprise,...,,,,,,,,,,OUTDOOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9130,None,None,E410-EA029D,172.25.173.195,default,cnPilot e410,W8WJ0HZK42GW,2021-11-05T12:03:10-05:00,onboarding,wifi-enterprise,...,,,,,,,,,,
9131,None,None,APOutdoor2SanPablo,172.25.215.149,default,cnPilot e510,W8XD00Q0ZX2V,2021-11-06T20:40:16-05:00,onboarding,wifi-enterprise,...,,,,,,,,,,
9132,None,None,APIndoorSanPablo,172.25.215.147,default,cnPilot e410,W8XA05C4XDJH,2021-11-06T20:32:19-05:00,onboarding,wifi-enterprise,...,,,,,,,,,,
9133,None,None,APOutdoorSanPablo,172.25.215.148,default,cnPilot e510,W8XD0HNP47KC,2021-11-06T20:38:43-05:00,onboarding,wifi-enterprise,...,,,,,,,,,,


In [132]:
#datos_api.dropna(axis=0, how='all', inplace=True)
datos_api.dropna(subset=['site_id'], inplace=True)
datos_api.fillna('', inplace=True)

In [133]:
datos_api

,country,description,name,ip,network,product,msn,registration_date,status,type,...,config.variables.VLAN_1_PREFIX6,config.variables.LOCATION,config.variables.SHUTDOWN_24,config.variables.CHANNEL_24,config.variables.SHUTDOWN_5,config.variables.CHANNEL_5,config.variables.POWER_24,config.variables.POWER_5,config.variables.DISPLAY_NAME,ap_group
0,CO,,44745-ZGYO227-AP-INT,172.28.118.179,SANTANDER,cnPilot e410,W8WJ037WHH74,2021-04-07T19:13:54-05:00,online,wifi-enterprise,...,,,,,,,,,,INDOOR-44745-ZGYO227
1,CO,,44745-ZGYO227-AP-EXT2,172.28.118.180,SANTANDER,cnPilot e510,W8VJ067FJH0D,2021-04-07T19:38:50-05:00,online,wifi-enterprise,...,,,,,,,,,,OUTDOOR
2,CO,,44745-ZGYO227-AP-EXT1,172.28.118.181,SANTANDER,cnPilot e510,W8VJ05X88C78,2021-04-08T10:50:15-05:00,online,wifi-enterprise,...,,,,,,,,,,OUTDOOR
3,CO,44747-ZGYO413-AP-INT,44747-ZGYO413-AP-INT,172.28.118.171,SANTANDER,cnPilot e410,W8WK06VL2ZGJ,2021-04-19T10:46:35-05:00,online,wifi-enterprise,...,,,,,,,,,,INDOOR_44747-ZGYO413
4,CO,44747-ZGYO413-AP-EXT-1,44747-ZGYO413-AP-EXT-1,172.28.118.172,SANTANDER,cnPilot e510,W8WF00HRR3VK,2021-04-19T10:46:34-05:00,online,wifi-enterprise,...,,,,,,,,,,OUTDOOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9130,,,E410-EA029D,172.25.173.195,default,cnPilot e410,W8WJ0HZK42GW,2021-11-05T12:03:10-05:00,onboarding,wifi-enterprise,...,,,,,,,,,,
9131,,,APOutdoor2SanPablo,172.25.215.149,default,cnPilot e510,W8XD00Q0ZX2V,2021-11-06T20:40:16-05:00,onboarding,wifi-enterprise,...,,,,,,,,,,
9132,,,APIndoorSanPablo,172.25.215.147,default,cnPilot e410,W8XA05C4XDJH,2021-11-06T20:32:19-05:00,onboarding,wifi-enterprise,...,,,,,,,,,,
9133,,,APOutdoorSanPablo,172.25.215.148,default,cnPilot e510,W8XD0HNP47KC,2021-11-06T20:38:43-05:00,onboarding,wifi-enterprise,...,,,,,,,,,,


In [134]:
datos_api.drop_duplicates(subset=['site_id','mac'],inplace=True)

In [135]:
datos_api

,country,description,name,ip,network,product,msn,registration_date,status,type,...,config.variables.VLAN_1_PREFIX6,config.variables.LOCATION,config.variables.SHUTDOWN_24,config.variables.CHANNEL_24,config.variables.SHUTDOWN_5,config.variables.CHANNEL_5,config.variables.POWER_24,config.variables.POWER_5,config.variables.DISPLAY_NAME,ap_group
0,CO,,44745-ZGYO227-AP-INT,172.28.118.179,SANTANDER,cnPilot e410,W8WJ037WHH74,2021-04-07T19:13:54-05:00,online,wifi-enterprise,...,,,,,,,,,,INDOOR-44745-ZGYO227
1,CO,,44745-ZGYO227-AP-EXT2,172.28.118.180,SANTANDER,cnPilot e510,W8VJ067FJH0D,2021-04-07T19:38:50-05:00,online,wifi-enterprise,...,,,,,,,,,,OUTDOOR
2,CO,,44745-ZGYO227-AP-EXT1,172.28.118.181,SANTANDER,cnPilot e510,W8VJ05X88C78,2021-04-08T10:50:15-05:00,online,wifi-enterprise,...,,,,,,,,,,OUTDOOR
3,CO,44747-ZGYO413-AP-INT,44747-ZGYO413-AP-INT,172.28.118.171,SANTANDER,cnPilot e410,W8WK06VL2ZGJ,2021-04-19T10:46:35-05:00,online,wifi-enterprise,...,,,,,,,,,,INDOOR_44747-ZGYO413
4,CO,44747-ZGYO413-AP-EXT-1,44747-ZGYO413-AP-EXT-1,172.28.118.172,SANTANDER,cnPilot e510,W8WF00HRR3VK,2021-04-19T10:46:34-05:00,online,wifi-enterprise,...,,,,,,,,,,OUTDOOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9130,,,E410-EA029D,172.25.173.195,default,cnPilot e410,W8WJ0HZK42GW,2021-11-05T12:03:10-05:00,onboarding,wifi-enterprise,...,,,,,,,,,,
9131,,,APOutdoor2SanPablo,172.25.215.149,default,cnPilot e510,W8XD00Q0ZX2V,2021-11-06T20:40:16-05:00,onboarding,wifi-enterprise,...,,,,,,,,,,
9132,,,APIndoorSanPablo,172.25.215.147,default,cnPilot e410,W8XA05C4XDJH,2021-11-06T20:32:19-05:00,onboarding,wifi-enterprise,...,,,,,,,,,,
9133,,,APOutdoorSanPablo,172.25.215.148,default,cnPilot e510,W8XD0HNP47KC,2021-11-06T20:38:43-05:00,onboarding,wifi-enterprise,...,,,,,,,,,,


### Formateando registration_date_new

In [136]:
try:
    datos_api['registration_date_new'] = (datos_api["registration_date"].str.split("T", n = 2, expand = True)[0])+' '+(datos_api["registration_date"].str.split("T", n = 2, expand = True)[1]).str.split("-", n = 1, expand = True)[0]
    datos_api.drop(columns=['registration_date'], inplace=True)
    datos_api.rename(columns={'registration_date_new': 'registration_date'}, inplace=True)
except:
    pass

In [137]:
#datos_api = datos_api.drop(datos_api[(datos_api['registration_date']<= fecha_max)].index)
datos_api.dropna(subset=['site_id'], inplace=True)
datos_api = datos_api.drop(datos_api[(datos_api['site_id']=='')].index)
datos_api = datos_api.drop(datos_api[(datos_api['network'].isin(['default','']))].index)

### Definiendo indice cambium-device-devices con fecha e insertando en ES
* se le quito la fecha_hoy (Solicitado por Napoleon 15/6/21 [GM] )


In [138]:
datos_api['network'].unique()

array(['SANTANDER', 'META', 'TOLIMA', 'HUILA', 'SUCRE',
       'NORTE DE SANTANDER', 'GUAJIRA', 'SUBA', 'RISARALDA', 'ATLANTICO',
       'CESAR', 'ANTIOQUIA', 'CORDOBA', 'CALDAS', 'CAQUETA', 'GUAVIARE',
       'ARCHIPIELAGO'], dtype=object)

In [139]:
indice = 'cambium_devicedevice_pruinge1' #+ '-' + fecha_hoy

In [140]:
use_these_keys = ['country', 'description', 'name', 'ip', 'network', 'product', 'msn',
                   #'software_version', 'inactive_software_version', 'last_reboot_reason','hardware_version',
                   'registration_date', 'status', 'type', 'tower',
                   'site', 'site_id', 'location.type','location.coordinates', 'mac', 'managed_account',
                   'status_time','config.version', 'config.variables.DEFAULT_GW',
                   'config.variables.DEFAULT_GW6', 'config.variables.DNS_SERVER6_1',
                   'config.variables.DNS_SERVER6_2', 'config.variables.DNS_SERVER_1',
                   'config.variables.DNS_SERVER_2', 'config.variables.VLAN_1_IP',
                   'config.variables.VLAN_1_IP6', 'config.variables.VLAN_1_MASK',
                   'config.variables.VLAN_1_MODE', 'config.variables.VLAN_1_MODE6',
                   'config.variables.VLAN_1_PREFIX6', 'config.variables.LOCATION',
                   'config.variables.SHUTDOWN_24', 'config.variables.CHANNEL_24',
                   'config.variables.SHUTDOWN_5', 'config.variables.CHANNEL_5',
                   'config.variables.POWER_24', 'config.variables.POWER_5',
                   'config.variables.DISPLAY_NAME', 'ap_group','@timestamp']
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

timestamp = datetime.now()
datos_api['@timestamp'] = timestamp.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{document['mac']}",
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(datos_api))
print("Fecha:",now,"- * Total Devices insertados/actualizados: ",salida[0])

Fecha: 2021-11-07 09:05:03.456799 - * Total Devices insertados/actualizados:  8515


## De los mismos datos se alimenta el indice cambium_d_mac_index

In [141]:
indice_mac = 'cambium_mac_pruinge' # + '-' + fecha_hoy

### se toman de datos_api solo aquellos con site_id diferente a vacio

In [124]:
datos_mac = datos_api[(datos_api['site_id']!='')][["country","description","name","ip","network","product","msn"
                          #,"software_version","inactive_software_version","last_reboot_reason","hardware_version"
                          ,"registration_date","status","type","site"
                          ,"location.type","location.coordinates","mac","site_id","managed_account"
                          ,"status_time"]]

In [125]:
use_these_keys = ['country','description','name','ip','network','product','msn'
                  #,'software_version','inactive_software_version','last_reboot_reason','hardware_version'
                  ,'registration_date','status','type','site'
                  ,'location.type','location.coordinates','mac','site_id','managed_account'
                  ,'status_time','@timestamp']
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

timestamp = datetime.now()
datos_mac['@timestamp'] = timestamp.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice_mac, 
                "_id": f"{document['mac']}",
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(datos_mac))
print("Fecha:",now,"- Total documentos insertados en indice_MAC: ",salida[0])

Fecha: 2021-11-04 11:43:58.217761 - Total documentos insertados en indice_MAC:  8257
